In [1]:
import numpy as np
from scipy.optimize import minimize

Data generation of 10 assets.

In [2]:
n_assests = 10
A = np.random.randn(n_assests, n_assests)
covariance_matrix = np.dot(A, A.transpose())
expected_returns = np.random.randn(n_assests)

Global minimum variance portfolio using closed form solution. 

In [3]:
inv_cov = np.linalg.inv(covariance_matrix)
ones = np.ones((len(expected_returns), 1))
w1 = (inv_cov @ ones ) / (ones.T @ inv_cov @ ones)
var1 = (w1.T @ covariance_matrix @ w1)
std1 = np.sqrt(var1)

std1

array([[0.43268911]])

Optimize portfolio variance with weights summing to one. It supports optional long-only and custom constraints. Returns optimal weights, expected return, and volatility.

In [5]:
def optimize_portfolio(covariance_matrix, expected_returns, 
                        extra_constraints=None, bounds=None):
    
    n = len(expected_returns)

    def portfolio_variance (w):
        return w.T @ covariance_matrix @ w

    w0 = np.ones(n) / n

    # normalize
    constraints = [{
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1 
    }]

    if extra_constraints is not None:
        constraints += extra_constraints

    if bounds is not None:
        bounds1 = [(bounds[0], bounds[1]) for _ in range(n_assests)]
    else:
        bounds1 = bounds

    result = minimize(portfolio_variance, w0, constraints=constraints, bounds=bounds1)

    if not result.success:
        raise ValueError("Optimization failed: " + result.message)

    w = result.x
    mu = w.T @ expected_returns
    var = w.T @ covariance_matrix @ w
    std = np.sqrt(var)

    return w, mu, std


Long-only, weights are at least 5%, aggregate weigths of first 3 assests below 20%.

In [6]:
n = len(expected_returns)

bounds = [0.0, 1.0]

min_weight = [{
    'type': 'ineq',
    'fun': lambda w, i=i: w[i] - 0.05} for i in range(n)
]

aggragate_sum = [{
    'type': 'ineq',
    'fun': lambda w: 0.20 - np.sum(w[0:3])
}]

extra_constraints = aggragate_sum + min_weight

optimize_portfolio(covariance_matrix=covariance_matrix,
                    expected_returns=expected_returns, 
                    bounds=bounds, extra_constraints=extra_constraints)

(array([0.06594064, 0.05      , 0.08405936, 0.21538766, 0.14102446,
        0.05      , 0.09754094, 0.05      , 0.16855592, 0.07749102]),
 -0.12887072646290582,
 0.49779678447461423)